## Load Pretrained VGG16 Model

In [6]:
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16

# Define image size
IMAGE_SHAPE = [224, 224, 3]

# Load VGG16 model (without the top layer)
vgg_model = VGG16(input_shape=IMAGE_SHAPE, weights='imagenet', include_top=False)

# Freeze all layers
for layer in vgg_model.layers:
    layer.trainable = False

## Modify the Model for Classification

In [7]:
# Flatten the VGG16 output
flatten_layer = Flatten()(vgg_model.output)

# Add a Dense (Fully Connected) Layer
fc_layer = Dense(128, activation='relu')(flatten_layer)

# Output layer (Binary Classification: Normal/Pneumonia)
output_layer = Dense(1, activation='sigmoid')(fc_layer)

# Create the final model
model = Model(inputs=vgg_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Load and Preprocess the data

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
TRAIN_DIR = "Pneumonia Detection Dataset/chest_xray/train"
TEST_DIR = "Pneumonia Detection Dataset/chest_xray/test"

# Image Augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_set = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(224, 224), batch_size=32, class_mode='binary')
test_set = test_datagen.flow_from_directory(TEST_DIR, target_size=(224, 224), batch_size=32, class_mode='binary')

Found 4401 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Train the Model

In [9]:
# Train the model
history = model.fit(train_set, validation_data=test_set, epochs=5, steps_per_epoch=len(train_set), validation_steps=len(test_set))

Epoch 1/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 583s 4s/step - accuracy: 0.8167 - loss: 0.5803 - val_accuracy: 0.8830 - val_loss: 0.3338
Epoch 2/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 576s 4s/step - accuracy: 0.9499 - loss: 0.1340 - val_accuracy: 0.9119 - val_loss: 0.2572
Epoch 3/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 577s 4s/step - accuracy: 0.9603 - loss: 0.1062 - val_accuracy: 0.8734 - val_loss: 0.4961
Epoch 4/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 581s 4s/step - accuracy: 0.9596 - loss: 0.1153 - val_accuracy: 0.9135 - val_loss: 0.2415
Epoch 5/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 599s 4s/step - accuracy: 0.9665 - loss: 0.0892 - val_accuracy: 0.9151 - val_loss: 0.3104


In [10]:
model.save("pneumonia_model.h5")